-----
# cartoGRAPHs | Main 

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

#### This notebook is divided into sections, which should be run sequentially.
+ Section 1 - where the network shall be defined (a networkx graph)
+ Section 2 - specify a layout algorithm and choose the dimensionality reduction
+ Section 3 - the visual parameters > setting node colors (dict: key=node ID from G.nodes() and value=any color value)
+ Section 4 - the layout visualization step 

-----

Please note: 
Layouts of Graphs with more than 5000 nodes can take a few minutes to run. 

-----

In [ ]:
from cartoGRAPHs import *

_____
# 1 | DEFINE NETWORK
_____

In [2]:
G = nx.scale_free_graph(n=1000)

l_nodefeatures = ['nodeID:'+str(i) for i in list(G.nodes())]
d_nodefeatures = dict(zip(G.nodes(),l_nodefeatures))

closeness = nx.closeness_centrality(G)
d_closeness = {}
for node, cl in sorted(closeness.items(), key = lambda x: x[1], reverse = 1):
    d_closeness[node] = round(cl,4)

_____
# 2 | LAYOUT METHOD 
_____
+ most important parameter to choose : layoutmethod e.g. 'local', 'global', 'importance', 'functional'
+ to see results, please run ONE of the following examples 
* STRUCTURAL LAYOUT
* FUNCTIONAL LAYOUT
* EXEMPLARY REAL NETWORK

### Example of Structural Layout

In [3]:
layout_method = 'global'

posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        )

posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = layout_method,
                        dimred_method='umap'
                        )

### Example of Functional Layout
This is an example of an artifically produced functional matrix.
The structure of the matrix shall be :
+ rows = number of nodes in the Graph
+ columns = number of features 

In [7]:
layout_method = 'functional'

scale = 1
val = 0
rows = len(list(G.nodes()))

feat_one = [(val) if i%3 else (scale) for i in range(rows)]
feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]

feat_matrix = np.vstack((feat_one,feat_two,feat_three))
FM = pd.DataFrame(feat_matrix)
FM.index = ['100','101','102']
FM=FM.T
FM.index = list(G.nodes())

In [ ]:
posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = FM
                        )

posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = FM
                        )

### Example of Precalculated Matrix Input 
This is highly recommended for example with large networks with more than 5000 nodes and 50k links. Here we use the human protein-protein interaction network for demonstration.
+ generating layouts with precalculated settings can take up to 2min. 

In [14]:
organism = 'human'
G = load_graph(organism)

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame.from_dict(d_centralities,orient='index')
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
d_closeness = dict(df_centralities['closeness'])
d_nodefeatures = load_genesymbols(G, organism)

netlayout = 'global' # examples: 'local' #'global' #'funct-bio' #'funct-mol' #'funct-cel' #'funct-dis'
DM_precalculated = load_datamatrix(G,organism, netlayout)

In [15]:
%%time 

layout_method = 'precalculated'
posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = DM_precalculated,
                        )

CPU times: user 1min 30s, sys: 8.45 s, total: 1min 38s
Wall time: 1min 4s


In [16]:
%%time 

layout_method = 'precalculated'
posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = DM_precalculated
                        )

CPU times: user 1min 52s, sys: 7.22 s, total: 1min 59s
Wall time: 47.1 s


____________
# 3 | SPECIFIC VISUAL NODE / EDGE SETTINGS
____________

#### SET NODES SIZES

In [4]:
scale_factor = 2
nodesize = list(draw_node_degree(G, scale_factor).values())
d_nodesize = dict(draw_node_degree(G, scale_factor))

#### SET NODE COLORS 

##### OPTION 1 : A DICTIONARY WITH node ID as keys and different values based on groups. 
Here an example with a color gradient is shown.

In [32]:
col_pal = 'YlOrRd'
d_nodecolors = color_nodes_from_dict(G, d_nodesize, palette = col_pal)

d_nodelegend = {}
c = 0
for i in range(len(set(d_nodecolors.values()))):
    d_nodelegend[i]='color'+str(c)
    c+=1

##### OPTION 2 : specific list of nodes to have a specific color; all others will be gray

In [11]:
# the list_of_nodes shall be chosen
sublist_of_nodes = list(G.nodes())[::2]
sublist_color = '#00FFFB'

d_nodecolors = color_nodes_from_list(G, l_nodes = sublist_of_nodes, col = sublist_color)
nodecolors = list(d_nodecolors.values())

first_groupname = 'A1'
second_groupname = 'A2'
d_nodelegend = dict(zip(set(d_nodecolors.values()),[first_groupname,second_groupname]))

_____ 
# 4 | INTERACTIVE VISUALIZATION + EXPORT TO VR
_____
Layout Maps to choose from: 
+ Portrait 2D 
+ Portrait 3D 
+ Topographic Map
+ Geodesic Map 

### 2D PORTRAIT

In [34]:
plot_2Dfigure(G, posG2D, 
              d_features = d_nodefeatures, 
              d_colors = d_nodecolors, 
              d_size = d_nodesize, 
              d_legend = d_nodelegend,
              path = 'output_plots/2Dlayouts/', 
              fname = '2Dportrait'+'_'+layout_method, 
              scheme = 'light',
              #with_edges = False
             )

'output_plots/2Dlayouts/2Dportrait_global.html'

#### EXPORT FOR VRNetzer 

In [ ]:
export_to_csv2D(path = 'output_plots/VRlayouts/', 
                layout_namespace = '2Dportrait'+'_'+layout_method, 
                posG2D, 
                nodecolors)

### 3D PORTRAIT 

In [33]:
plot_3Dfigure(G, posG3D, 
              d_features = d_nodefeatures, 
              d_colors = d_nodecolors, 
              d_size = d_nodesize, 
              d_legend = d_nodelegend,
              path = 'output_plots/3Dlayouts/', 
              fname = '3Dportrait'+'_'+layout_method, 
              scheme = 'dark',
              #with_edges = False
             )

'output_plots/3Dlayouts/3Dportrait_global.html'

#### EXPORT FOR VRNetzer 

In [21]:
export_to_csv3D(path = 'output_plots/VRlayouts/', 
                layout_namespace = '3Dportrait'+layout_method, 
                posG3D, 
                nodecolors)

### TOPOGRAPHIC MAP

In [23]:
# d_z = a dictionary with keys=G.nodes and values=any int/float assigned to a node
 
z_list = [np.random.random() for i in range(0, len(list(G.nodes())))]
d_z = dict(zip(list(G.nodes()),z_list))
posG_topographic = layout_topographic(posG2D, d_z)

plot_3Dfigure(G, posG_topographic, 
              d_features = d_nodefeatures, 
              d_colors = d_nodecolors, 
              d_size = d_nodesize, 
              d_legend = d_nodelegend,
              path = 'output_plots/3Dlayouts/', 
              fname = 'Topographic'+layout_method, 
              scheme = 'light')

'output_plots/3Dlayouts/Topographicfunctional.html'

#### EXPORT FOR VRNetzer 

In [20]:
export_to_csv3D(path = 'output_plots/VRlayouts/',
                layout_namespace = 'Topographic'+layout_method, 
                posG = posG_topographic, 
                colors = nodecolors)

### GEODESIC MAP

In [ ]:
# d_rad = a dictionary with keys=G.nodes and values=any radius assigned to each node

rad_list = [np.random.randint(1,4) for i in range(0, len(list(G.nodes())))]
d_rad = dict(zip(list(G.nodes()), rad_list))
posG_geodesic = layout_geodesic(G, d_rad)

plot_3Dfigure(G, posG_geodesic, 
              d_features = d_nodefeatures, 
              d_colors = d_nodecolors, 
              d_size = d_nodesize, 
              d_legend = d_nodelegend, 
              path = 'output_plots/3Dlayouts/', 
              fname = 'Geodesic'+layout_method, 
              scheme = 'light')

#### EXPORT FOR VRNetzer 

In [31]:
export_to_csv3D(path = 'output_plots/VRlayouts/', 
                layout_namespace = 'Geodesic'+layout_method, 
                posG = posG_geodesic, 
                colors = nodecolors)